## 1. 각종 모듈 불러오기

In [1]:
import io
import requests
import pandas as pd
import urllib.request
import zipfile 
import xml.etree.ElementTree as et 
import json
from urllib import request
from zipfile import ZipFile
import ssl, os
import time
from random import *

In [2]:
api_key = '1b9dbd41324b867fb29eaadc23b06240b1ae40b0' #발급 받은 api_key

## 2. 공시 정보를 파일로 불러온다

In [3]:
# 공시정보 - 다운로드

context = ssl._create_unverified_context()

url = "https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key="+api_key
print("요청주소 : " + url)

data = request.urlopen(url, context=context)
filename = data.info().get_filename()
print("파일명 : " + filename)

with open(filename, 'wb') as f:
    f.write(data.read())
    f.close

print("다운로드 완료.")

with ZipFile(filename, 'r') as zipObj:
    zipObj.extractall('./') # 현재 디렉토리에 압축을 해제

if os.path.isfile(filename):
    os.remove(filename) # 원본 압축파일 삭제

요청주소 : https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=1b9dbd41324b867fb29eaadc23b06240b1ae40b0
파일명 : CORPCODE.zip
다운로드 완료.


## 3. 필요한 정보를 선별해 데이터 프레임으로 만든다 (기업이름,고유코드,주식코드)

In [4]:
import xml.etree.ElementTree as ET

#추가할 list
company_name_list = [] 
company_code_list = []
company_stock_code_list = []

tree = ET.parse('CORPCODE.xml') # CORPCODE.xml을 파싱하여 tree에 저장 
root = tree.getroot()

for company in root.findall('list'):
    
    company_name = company.find('corp_name').text #기업이름
    company_code = company.find('corp_code').text #기업코드
    company_stock_code = company.find('stock_code').text #주식코드
    
    company_name_list.append(company_name)
    company_code_list.append(company_code)
    company_stock_code_list.append(company_stock_code)

In [5]:
len(company_stock_code_list)

81145

In [6]:
df_company = pd.DataFrame({'cp_name': company_name_list,'cp_code': company_code_list,'cp_stock_code': company_stock_code_list})

In [7]:
df_company[df_company['cp_stock_code']!=' '] #주식코드 공백 기업 제거 : only 상장기업

,cp_name,cp_code,cp_stock_code
2015,한빛네트,00260985,036720
2027,엔플렉스,00264529,040130
2028,동서정보기술,00358545,055000
2792,애드모바일,00231567,032600
3902,씨모스,00247939,037600
...,...,...,...
81114,제넥신,00545929,095700
81130,한국타이어앤테크놀로지,00937324,161390
81131,메디프론,00391197,065650
81132,캔서롭,00995993,180400


In [8]:
df_crp = df_company[df_company['cp_stock_code']!=' '] #데이터프레임으로 저장

In [9]:
crp_code = df_crp["cp_code"]
#start_date = "19900101"

## 4. 각 회사의 감사 보고서 주소를 불러온다

In [10]:
crp_F001_list = []

In [12]:
#데이터 url

#2016년 1월 1일부터 설정 - 너무 오래전 부터 가져 오면 중간에 테이블 형식이 바뀌어서 테이블 규격이 어긋남

#3~6분 소요 예상
for crp_code in df_crp["cp_code"]:
    url ='https://opendart.fss.or.kr/api/list.json?crtfc_key='+api_key+'&corp_code='+crp_code+'&pblntf_detail_ty=F001&bgn_de=20160101&page_count=100'
    r = requests.get(url)
    if(r.json()['status'] == '000'):
        company_data = r.json()['list']
        crp_F001_list.append(company_data)
   



ConnectionError: HTTPSConnectionPool(host='opendart.fss.or.kr', port=443): Max retries exceeded with url: /api/list.json?crtfc_key=1b9dbd41324b867fb29eaadc23b06240b1ae40b0&corp_code=00260985&pblntf_detail_ty=F001&bgn_de=20160101&page_count=100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001F8C7396248>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [ ]:
crp_F001_list[1][0] #데이터가 잘 들어갔는지 확인

In [ ]:
crp_F001_list[:] #데이터가 잘 들어갔는지 확인

In [ ]:
len(crp_F001_list)

In [ ]:
#리스트를 데이터프레임으로 변경

df_all = pd.DataFrame()
for i in range(len(crp_F001_list)):
    df = pd.DataFrame(crp_F001_list[i])
    df_all = pd.concat([df_all,df])

In [ ]:
df_all = df_all.reset_index()
df_all.drop(["index"],1,inplace=True)
df_all

## 5. 불러온 주소로 외부감사 실시내용의 테이블을 가져 온다

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
# chrome driver 와 selenium 설정
path = "./chromedriver.exe" #버전에 맞는 크롬 드라이버 파일 다운로드 후 경로로 설정해야 한다 
driver = webdriver.Chrome(path)

#여기
#html = driver.page_source
#soup = BeautifulSoup(html, 'html.parser')

In [ ]:
df_audit = pd.DataFrame()

for row,col in df_all.iterrows():
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo='+str(col['rcept_no'])

    driver.get(url)

    
    tags = driver.find_elements_by_class_name('x-tree-node-anchor')
    tag = tags[-1] #가장 마지막 테이블에 외부감사 실시내용 링크
    tag.find_element_by_tag_name("span").click()    
           
    ifrm_tags = soup.find_all('iframe')
      
    
    full_url = 'http://dart.fss.or.kr' + ifrm_tags[0]['src']
    
    
    df_table = pd.read_html(full_url,header=[0,1])    
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    
    if(len(df_table)>2):
        for i in range(len(df_table)):
            if(str(df_table[i]).find('감사참여자') > -1):
                df = df_table[i]        

        df['corp_name'] = col['corp_name']
        
        split_word = col['report_nm'].split('(')
        audit_period = split_word[1].split(')')
        
        audit_year_month = audit_period[0].split('.')
        audit_year = audit_year_month[0]
        audit_month = audit_year_month[1]
                
        df['fyear'] = audit_year
        df['fmonth'] = audit_month
        
        if(len(df_audit) != 0):
            df.drop(0,inplace=True)
        df_audit = pd.concat([df_audit,df])
    else:
        continue
    #time.sleep(1) #타임을 1초씩 걸면 강제 종료 당함
    time.sleep(randint(2,4))

In [66]:
df

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,01418260,폭스소프트,354230,N,감사보고서 (2019.12),20200403002619,회계법인창천,20200403,


In [163]:
df_audit

corp_name fmonth fyear  감사업무 담당 회계사                                   \
                               담당이사(업무수행이사) 담당이사(업무수행이사).1 등록공인회계사 등록공인회계사.1   
0   코리언일랙트로닉스파워소스     12  2015           당기             전기      당기        전기   
1   코리언일랙트로닉스파워소스     12  2015            1              1       1         1   
2   코리언일랙트로닉스파워소스     12  2015            -              -       -         -   
3   코리언일랙트로닉스파워소스     12  2015            2              2       8         7   
4   코리언일랙트로닉스파워소스     12  2015            2              2       8         7   
..            ...    ...   ...          ...            ...     ...       ...   
3            글라소울     12  2015            6              -     198         -   
4            글라소울     12  2015            6              -     198         -   
1           아이디에스     12  2016            -            NaN       -       NaN   
2           아이디에스     12  2016           64            NaN     128       NaN   
3           아이디에스     12  2016           64            NaN     128       NaN   

                      감사참여자 ＼ 인원수 및 시간                    건설계약 등수주산업전문가  \
   수습공인회계사 수습공인회계사.1  감사참여자 ＼ 인원수 및 시간 감사참여자 ＼ 인원수 및 시간.1 건설계약 등수주산업전문가   
0       당기        전기  감사참여자 ＼ 인원수 및 시간   감사참여자 ＼ 인원수 및 시간            당기   
1        -         -            투입 인원수             투입 인원수             -   
2        -         -              투입시간             분ㆍ반기검토             -   
3        -         -              투입시간                 감사             -   
4        -         -              투입시간                 합계             -   
..     ...       ...               ...                ...           ...   
3        -         -              투입시간                 감사             -   
4        -         -              투입시간                 합계             -   
1        -       NaN              투입시간             분ㆍ반기검토           NaN   
2        -       NaN              투입시간                 감사           NaN   
3        -       NaN              투입시간                 합계           NaN   

                   전산감사ㆍ세무ㆍ가치평가 등 전문가                      품질관리검토자(심리실 등)  \
   건설계약 등수주산업전문가.1 전산감사ㆍ세무ㆍ가치평가 등 전문가 전산감사ㆍ세무ㆍ가치평가 등 전문가.1 품질관리검토자(심리실 등)   
0               전기                 당기                   전기             당기   
1                -                  -                    -              1   
2                -                  -                    -              -   
3                -                  -                    -              1   
4                -                  -                    -              1   
..             ...                ...                  ...            ...   
3                -                  -                    -              4   
4                -                  -                    -              4   
1              NaN                  -                  NaN              -   
2              NaN                  -                  NaN              8   
3              NaN                  -                  NaN              8   

                      합계       
   품질관리검토자(심리실 등).1   합계 합계.1  
0                전기   당기   전기  
1                 1    3    3  
2                 -    -    -  
3                 1   11   10  
4                 1   11   10  
..              ...  ...  ...  
3                 -  208    -  
4                 -  208    -  
1               NaN    -  NaN  
2               NaN  200  NaN  
3               NaN  200  NaN  

[83 rows x 19 columns]

In [164]:
df_audit.to_csv("audit_final3.csv",encoding="utf-8-sig",index=False)